# Laboratorio 0 — Preparación del entorno y buenas prácticas

Este cuaderno define convenciones (reproducibilidad, estructura de experimentos y métricas) que se reutilizarán en el resto de prácticas.

## Objetivos
- Verificar el entorno (versiones, GPU si aplica).
- Establecer semillas y una estructura mínima de experimento.
- Adoptar un flujo de trabajo repetible: datos → modelo → evaluación → registro.

## Requisitos
- Python 3.10+ recomendado
- Jupyter
- numpy, pandas, matplotlib, scikit-learn


## 1) Comprobación del entorno

Ejecuta la siguiente celda para ver versiones. Si alguna librería falta, instala con `pip install ...` (idealmente dentro de un entorno virtual).


In [1]:
import sys, platform
import numpy as np
import pandas as pd
import sklearn
import matplotlib

print("Python:", sys.version.split()[0])
print("Plataforma:", platform.platform())
print("NumPy:", np.__version__)
print("Pandas:", pd.__version__)
print("Scikit-learn:", sklearn.__version__)
print("Matplotlib:", matplotlib.__version__)


Python: 3.13.9
Plataforma: Windows-11-10.0.26100-SP0
NumPy: 2.3.4
Pandas: 2.3.3
Scikit-learn: 1.8.0
Matplotlib: 3.10.7


## 2) Reproducibilidad

En ciencia de datos es habitual obtener resultados distintos entre ejecuciones por aleatoriedad (barajado de datos, inicialización de pesos, etc.). 
Aquí fijamos semillas y definimos utilidades para evaluar de forma consistente.


In [2]:
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

print("Semilla fijada:", SEED)


Semilla fijada: 42


## 3) Utilidades de evaluación

Definimos dos funciones prácticas: una para dividir conjuntos y otra para reportar métricas de forma estándar.
Más adelante se extenderán para CV y NLP.


In [3]:
from dataclasses import dataclass
from typing import Dict, Any, Tuple

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, f1_score

def split(X, y, test_size=0.2, seed=42):
    return train_test_split(X, y, test_size=test_size, random_state=seed)

def regression_report(y_true, y_pred) -> Dict[str, float]:
    return {
        "MAE": float(mean_absolute_error(y_true, y_pred)),
        "RMSE": float(mean_squared_error(y_true, y_pred, squared=False)),
        "R2": float(r2_score(y_true, y_pred)),
    }

def classification_report_simple(y_true, y_pred) -> Dict[str, float]:
    return {
        "Accuracy": float(accuracy_score(y_true, y_pred)),
        "F1 (macro)": float(f1_score(y_true, y_pred, average="macro")),
    }


## 4) Registro mínimo de experimentos

No hace falta una plataforma compleja para empezar; basta con guardar:
- configuración (modelo/hiperparámetros),
- métricas,
- fecha/hora y semilla.

Esto permite comparar versiones sin perderse.


In [4]:
import json
from datetime import datetime
from pathlib import Path

# 1️⃣ Cargas o creas el dataset AQUÍ
X = pd.DataFrame({
    "edad": [25, 32, 41],
    "salario": [22000, 30000, 47000],
    "compra": [0, 1, 1]
})

def log_experiment(name: str, config: Dict[str, Any], metrics: Dict[str, Any], out_dir="experimentos"):
    Path(out_dir).mkdir(exist_ok=True)
    payload = {
        "name": name,
        "timestamp": datetime.now().isoformat(timespec="seconds"),
        "seed": SEED,
        "config": config,
        "metrics": metrics,
    }
    path = Path(out_dir) / f"{name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    path.write_text(json.dumps(payload, ensure_ascii=False, indent=2))
    return str(path)

print("Listo. Usa log_experiment(...) en los laboratorios.")


Listo. Usa log_experiment(...) en los laboratorios.


## Ejercicios

### Ejercicio 1: Entorno reproducible
Crea un entorno virtual (venv/conda) e instala las librerías necesarias. Exporta un `requirements.txt` mínimo para estos laboratorios.

**Entregables**
- Comando(s) usados
- Archivo `requirements.txt`

**Comandos:**
- py -m venv venv
- venv\Scripts\activate
- pip install pandas numpy scikit-learn
- pip freeze > requirements.txt

**Criterios de evaluación**
- El entorno se crea y activa correctamente
- El requirements incluye versiones o, como mínimo, librerías clave
- Se documenta cómo reproducirlo

### Ejercicio 2: Plantilla de experimento
Extiende `log_experiment` para que también guarde el tamaño del dataset y el nombre de las columnas/variables.

**Entregables**
- Función modificada
- Ejemplo de uso con un dataset pequeño

**Pistas**
- `X.shape` y `list(X.columns)` si trabajas con pandas

**Criterios de evaluación**
- Registra información adicional útil
- No rompe compatibilidad con el uso anterior
- El JSON queda legible
